### 1. 建立資料夾

因為我之後有一份報告想嘗試用RAG作一個網站的小客服，所以我先用我們[實驗室](https://ila-lab.github.io/)的網站來試試練習

In [ ]:

import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx, .html, .js, .py 檔案放到這個資料夾中： {upload_dir}")

請將你的 .txt, .pdf, .docx, .html, .js, .py 檔案放到這個資料夾中： uploaded_docs


<font color="red">請手動上傳檔案再繼續。</font>

### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx faiss-cpu
!pip install -U sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

### 3. 依 Google 建議加入 EmbeddingGemma 前綴

Google 建議, 在文本部份的 Embedding 要用以下格式:

    title: {title|none} | text: ...

而問題 Query 要用:

    Query：task: search result | query: ...

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:

class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",   # HF 上的官方模型
            encode_kwargs={"normalize_embeddings": True},  # 一般檢索慣例
            **kwargs
        )

    def embed_documents(self, texts):
        # 文件向量：title 可用 "none"，或自行帶入檔名/章節標題以微幅加分
        texts = [f'title: none | text: {t}' for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 查詢向量：官方建議的 Retrieval-Query 前綴
        return super().embed_query(f'task: search result | query: {text}')

### 4. 載入文件

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 14.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=2f34a8bf524c6f96c2cb916b608934c462c75fe9e5306025c69364317cb6b286
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
import os

# 引入適合 HTML, JS, Python 檔案的載入器
# 使用 UnstructuredFileLoader 處理多種文件（需要安裝相關依賴）
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader, UnstructuredFileLoader

upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
# 更新提示訊息
print(f"請將你的 .txt, .pdf, .docx, .html, .js, .py 檔案放到這個資料夾中： {upload_dir}")

# ... (EmbeddingGemmaEmbeddings 類別保持不變)

folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)

    # 略過隱藏檔案 (如 .DS_Store)
    if file.startswith('.'):
        continue

    # 針對不同檔案類型使用適當的載入器
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    # 新增對 .html, .js, .py 檔案的處理
    elif file.endswith((".html", ".js", ".py")):
        # UnstructuredFileLoader 可以處理多種純文字和結構化文件
        # 它會自動嘗試提取內容，特別適合程式碼和結構化檔案
        loader = UnstructuredFileLoader(path)
    else:
        continue

    try:
        documents.extend(loader.load())
    except Exception as e:
        print(f"處理檔案 {file} 時發生錯誤: {e}")
        continue


請將你的 .txt, .pdf, .docx, .html, .js, .py 檔案放到這個資料夾中： uploaded_docs


### 5. 建立向量資料庫

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HuggingFace')

In [ ]:
from huggingface_hub import login
login(token=hf_token)

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

/tmp/ipython-input-3683757148.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db.zip faiss_db

  adding: faiss_db/ (stored 0%)
  adding: faiss_db/index.faiss (deflated 7%)
  adding: faiss_db/index.pkl (deflated 61%)


In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")

✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。
